Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Logistic Regression

In [57]:
batch_size = 128
beta = 0.05
num_steps = 10000
train_subset = 50000

graph = tf.Graph()

with graph.as_default():
    tf_train_batch = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_test_set = tf.constant(test_dataset)
    tf_valid_set = tf.constant(valid_dataset)
    
    tf_weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    tf_biases = tf.Variable(tf.truncated_normal([num_labels]))
    tf_logits = tf.matmul(tf_train_batch, tf_weights) + tf_biases
    
    tf_old_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=tf_logits))
    tf_regularizer = tf.nn.l2_loss(tf_weights)  
    tf_loss = tf.reduce_mean(tf_old_loss + beta * tf_regularizer)
    
    tf_optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(tf_loss)
                         
    tf_train_prediction = tf.nn.softmax(tf_logits)
    tf_valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_set, tf_weights)+tf_biases)
    tf_test_prediction = tf.nn.softmax(tf.matmul(tf_test_set, tf_weights)+tf_biases)

In [58]:

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        indexes = np.random.randint(low=0, high=train_subset, size=batch_size)
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        #batch_data = train_dataset[offset:(offset + batch_size), :]
        #batch_labels = train_labels[offset:(offset + batch_size), :]
        batch_data = train_dataset[indexes]
        batch_labels = train_labels[indexes]
        
        f_dict = {tf_train_batch: batch_data, tf_train_labels : batch_labels}
        _, l, train_pred = session.run([tf_optimizer, tf_loss, tf_train_prediction], feed_dict = f_dict)
        
        if(step % 2000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(train_pred, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(tf_valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(tf_test_prediction.eval(), test_labels))
            print('\n')

Minibatch loss at step 0: 167.539490
Minibatch accuracy: 9.4%
Validation accuracy: 9.1%
Test accuracy: 8.9%


Minibatch loss at step 2000: 0.944739
Minibatch accuracy: 77.3%
Validation accuracy: 80.7%
Test accuracy: 87.8%


Minibatch loss at step 4000: 0.791204
Minibatch accuracy: 83.6%
Validation accuracy: 80.8%
Test accuracy: 88.0%


Minibatch loss at step 6000: 0.843822
Minibatch accuracy: 82.0%
Validation accuracy: 81.1%
Test accuracy: 88.0%


Minibatch loss at step 8000: 1.086287
Minibatch accuracy: 72.7%
Validation accuracy: 81.0%
Test accuracy: 88.0%




Neural Network

In [53]:
batch_size = 128
beta = 0.05
num_steps = 10000
train_subset = 200000
h_size = 1024

In [61]:
graph = tf.Graph()

with graph.as_default():
    tf_train_d = tf.placeholder(tf.float32, shape=[batch_size, image_size*image_size])
    tf_train_l = tf.placeholder(tf.float32, shape=[batch_size, num_labels])
    
    tf_valid_d = tf.constant(valid_dataset)
    tf_test_d = tf.constant(test_dataset)
    
    #Layer 1
    l1_w = tf.Variable(tf.truncated_normal([image_size*image_size, h_size]))
    l1_b = tf.Variable(tf.zeros([h_size]))
    
    #Layer 2
    l2_w = tf.Variable(tf.truncated_normal([h_size, num_labels]))
    l2_b = tf.Variable(tf.zeros([num_labels]))
    
    #Training computation
    relu_layer = tf.nn.relu(tf.matmul(tf_train_d, l1_w)+l1_b)
    l2_out = tf.matmul(relu_layer, l2_w)+l2_b
    
    #Loss, regularization and optimization
    tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_l, logits=l2_out))
    l1_regularizer = tf.nn.l2_loss(l1_w)
    l2_regularizer = tf.nn.l2_loss(l2_w)
    regularized_loss = tf_loss + beta*(l1_regularizer + l2_regularizer)
    
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(regularized_loss)
    
    #Prediction outputs
    g_train_pred = tf.nn.softmax(l2_out)
    
    #Validation computation
    relu_layer = tf.nn.relu(tf.matmul(tf_valid_d, l1_w)+l1_b)
    l2_out = tf.matmul(relu_layer, l2_w)+l2_b
    g_valid_pred = tf.nn.softmax(l2_out)
    
    #Test computation
    relu_layer = tf.nn.relu(tf.matmul(tf_test_d, l1_w)+l1_b)
    l2_out = tf.matmul(relu_layer, l2_w)+l2_b
    g_test_pred = tf.nn.softmax(l2_out)
    

In [62]:

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        indexes = np.random.randint(low=0, high=train_subset, size=batch_size)
        batch_data = train_dataset[indexes]
        batch_labels = train_labels[indexes]
        
        f_dict = {tf_train_d: batch_data, tf_train_l : batch_labels}
        _, l, train_pred = session.run([optimizer, regularized_loss, g_train_pred], feed_dict = f_dict)
        
        if(step % 2000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(train_pred, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(g_valid_pred.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(g_test_pred.eval(), test_labels))
            print('\n')

Minibatch loss at step 0: 16156.357422
Minibatch accuracy: 9.4%
Validation accuracy: 19.2%
Test accuracy: 20.4%


Minibatch loss at step 2000: 1.835971
Minibatch accuracy: 76.6%
Validation accuracy: 81.4%
Test accuracy: 88.1%


Minibatch loss at step 4000: 1.002996
Minibatch accuracy: 84.4%
Validation accuracy: 81.3%
Test accuracy: 88.1%


Minibatch loss at step 6000: 0.989386
Minibatch accuracy: 84.4%
Validation accuracy: 81.5%
Test accuracy: 88.3%


Minibatch loss at step 8000: 0.897653
Minibatch accuracy: 88.3%
Validation accuracy: 81.5%
Test accuracy: 88.3%




---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [41]:
batch_size = 128
beta = 0.05
num_steps = 10000
train_subset = 50000
train_subset = 300

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        indexes = np.random.randint(low=0, high=train_subset, size=batch_size)
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        #batch_data = train_dataset[offset:(offset + batch_size), :]
        #batch_labels = train_labels[offset:(offset + batch_size), :]
        batch_data = train_dataset[indexes]
        batch_labels = train_labels[indexes]
        
        f_dict = {tf_train_batch: batch_data, tf_train_labels : batch_labels}
        _, l, train_pred = session.run([tf_optimizer, tf_loss, tf_train_prediction], feed_dict = f_dict)
        
        if(step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(train_pred, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(tf_valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(g_test_pred.eval(), test_labels))
            print('\n')

Minibatch loss at step 0: 168.229919
Minibatch accuracy: 8.6%
Validation accuracy: 9.4%


Minibatch loss at step 1000: 1.488619
Minibatch accuracy: 98.4%
Validation accuracy: 72.7%


Minibatch loss at step 2000: 0.533384
Minibatch accuracy: 99.2%
Validation accuracy: 74.5%


Minibatch loss at step 3000: 0.525990
Minibatch accuracy: 100.0%
Validation accuracy: 74.5%


Minibatch loss at step 4000: 0.524957
Minibatch accuracy: 97.7%
Validation accuracy: 74.2%


Minibatch loss at step 5000: 0.564642
Minibatch accuracy: 98.4%
Validation accuracy: 74.5%


Minibatch loss at step 6000: 0.544626
Minibatch accuracy: 97.7%
Validation accuracy: 74.1%


Minibatch loss at step 7000: 0.547053
Minibatch accuracy: 99.2%
Validation accuracy: 74.3%


Minibatch loss at step 8000: 0.526971
Minibatch accuracy: 100.0%
Validation accuracy: 74.5%


Minibatch loss at step 9000: 0.555428
Minibatch accuracy: 97.7%
Validation accuracy: 74.3%




---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [90]:
batch_size = 128
beta = 0.05
num_steps = 3001
print_period = 1000
train_subset = 200000
hidden_size = 1024
decaying_learning_rate = False

In [91]:
graph = tf.Graph()

with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[batch_size, image_size*image_size])
    y = tf.placeholder(tf.float32, shape=[batch_size, num_labels])
    dropout_keep_prob = tf.placeholder(tf.float32)
    
    v_x = tf.constant(valid_dataset)
    t_x = tf.constant(test_dataset)
    
    #Hidden layer
    w1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_size]))
    b1 = tf.Variable(tf.zeros([hidden_size]))
    
    #Output layer
    w2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    #Training computation
    #Computes dropout connections
    relu_layer = tf.nn.relu(tf.matmul(x, w1) + b1)
    relu_dropout = tf.nn.dropout(relu_layer, dropout_keep_prob)
    logits = tf.matmul(relu_dropout, w2)+b2
    
    #Loss and regularization
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    l1_reg = tf.nn.l2_loss(w1)
    l2_reg = tf.nn.l2_loss(w2)
    reg_loss = loss + beta * (l1_reg + l2_reg)
    
    if(decaying_learning_rate):
        global_step = tf.Variable(0)
        initial_learning_rate = 0.5
        learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, 100000, 0.96, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(reg_loss, global_step=global_step)
    else:
        optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Training predictions
    train_y = tf.nn.softmax(logits)
    
    #Validation predictions
    relu_layer = tf.nn.relu(tf.matmul(v_x, w1)+b1)
    logits = tf.matmul(relu_layer, w2)+b2
    val_y = tf.nn.softmax(logits)
    
    #Testing predictions
    relu_layer = tf.nn.relu(tf.matmul(t_x, w1)+b1)
    logits = tf.matmul(relu_layer, w2)+b2
    test_y = tf.nn.softmax(logits)

In [92]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        indexes = np.random.randint(low=0, high=train_subset, size=batch_size)

        batch_data = train_dataset[indexes]
        batch_labels = train_labels[indexes]
        
        f_dict = {x: batch_data, y : batch_labels, dropout_keep_prob : 0.5}
        _, l, train_pred = session.run([optimizer, reg_loss, train_y], feed_dict = f_dict)
        
        if(step % print_period == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(train_pred, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(val_y.eval(), valid_labels))
            print("Testing accuracy: %.1f%%" % accuracy(test_y.eval(), test_labels))
            print('\n')

Minibatch loss at step 0: 16176.153320
Minibatch accuracy: 6.2%
Validation accuracy: 24.6%
Testing accuracy: 26.5%


Minibatch loss at step 1000: 15507.916992
Minibatch accuracy: 80.5%
Validation accuracy: 83.0%
Testing accuracy: 90.4%


Minibatch loss at step 2000: 15415.918945
Minibatch accuracy: 79.7%
Validation accuracy: 83.4%
Testing accuracy: 90.5%


Minibatch loss at step 3000: 15365.236328
Minibatch accuracy: 77.3%
Validation accuracy: 83.5%
Testing accuracy: 90.9%




---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
